In [1]:
# -*- coding: utf-8 -*-
"""

Dictionnaires décrivants les transposés et symétries de relations, 
ainsi que les listes de relations obtenues avec les compositions de base
dans le tableau donné en TD

"""

transpose = {
    '<':'>',
    '>':'<',
    'e':'et',
    's':'st',
    'et':'e',
    'st':'s',
    'd':'dt',
    'm':'mt',
    'dt':'d',
    'mt':'m',
    'o':'ot',
    'ot':'o',
    '=':'='                 
    }
 
symetrie = {
    '<':'>',
    '>':'<',
    'e':'s',
    's':'e',
    'et':'st',
    'st':'et',
    'd':'d',
    'm':'mt',
    'dt':'dt',
    'mt':'m',
    'o':'ot',
    'ot':'o',
    '=':'='
    }            
             
compositionBase = {
        
        
        ('<','<'):{'<'},
        ('<','m'):{'<'},
        ('<','o'):{'<'},
        ('<','et'):{'<'},
        ('<','s'):{'<'},
        ('<','d'):{'<','m','o','s','d'},
        ('<','dt'):{'<'},
        ('<','e'):{'<','m','o','s','d'},
        ('<','st'):{'<'},
        ('<','ot'):{'<','m','o','s','d'},
        ('<','mt'):{'<','m','o','s','d'},
        ('<','>'):{'<','>','m','mt','o','ot','e','et','s','st','d','dt','='},
        ('m','m'):{'<'},
        ('m','o'):{'<'},
        ('m','et'):{'<'},
        ('m','s'):{'m'},
        ('m','d'):{'o','s','d'},
        ('m','dt'):{'<'},
        ('m','e'):{'o','s','d'},
        ('m','st'):{'m'},
        ('m','ot'):{'o','s','d'},
        ('m','mt'):{'e','et','='},
        ('o','o'):{'<','m','o'},
        ('o','et'):{'<','m','o'},
        ('o','s'):{'o'},
        ('o','d'):{'o','s','d'},
        ('o','dt'):{'<','m','o','et','dt'},
        ('o','e'):{'o','s','d'},
        ('o','st'):{'o','et','dt'},
        ('o','ot'):{'o','ot','e','et','d','dt','st','s','='},
        ('s','et'):{'<','m','o'},
        ('s','s'):{'s'},
        ('s','d'):{'d'},
        ('s','dt'):{'<','m','o','et','dt'},
        ('s','e'):{'d'},
        ('s','st'):{'s','st','='},
        ('et','s'):{'o'},
        ('et','d'):{'o','s','d'},
        ('et','dt'):{'dt'},
        ('et','e'):{'e','et','='},
        ('d','d'):{'d'},
        ('d','dt'):{'<','>','m','mt','o','ot','e','et','s','st','d','dt','='},
        ('dt','d'):{'o','ot','e','et','d','dt','st','s','='}
                           
                   }

In [163]:
def transposeSet(S):
    return set([transpose[i] for i in S])

def symetrieSet(S):
    return set([symetrie[i] for i in S])
        

In [164]:
l= ['<','>','et','s']
transposeSet(l)

{'<', '>', 'e', 'st'}

In [167]:
def compose(r1,r2):
    if r1 == '=':
        return {r2}
    if r2 == '=':
        return {r1}
    if (r1,r2) in compositionBase.keys():
        return compositionBase[(r1,r2)]
    if (transpose[r2],transpose[r1]) in compositionBase.keys():
        return transposeSet(compositionBase[(transpose[r2],transpose[r1])])
    if (symetrie[r1],symetrie[r2]) in compositionBase.keys():
        return symetrieSet(compositionBase[(symetrie[r1],symetrie[r2])])
    return symetrieSet(transposeSet(compositionBase[(transpose[symetrie[r2]],transpose[symetrie[r1]])]))

In [168]:
print(compose('ot','m'))

set(['et', 'dt', 'o'])


In [169]:
def compose_set(S1,S2):
    return  set([e for r2 in S2 for r1 in S1 for e in compose(r1,r2) ])

In [170]:
compose_set({'<','='},{'o','m'})

{'<', 'm', 'o'}

In [211]:
ALL_RELATIONS = set(['<', '>', 'e', 's', 'et', 'st', 'd', 'm', 'dt', 'mt', 'o',
                    'ot','='])
class Graphe:
    def __init__(self):
        self.noeuds = set()
        self.relations = dict()
    def get_noueds(self):
        return self.noeuds
    def getRelations(self, i, j):
        """ Retourne l'ensemble des relations entre les noeuds `i` et `j`. """
        global ALL_RELATIONS

        try:
            return self.relations[i, j]
        except KeyError:
            return ALL_RELATIONS
    def addNoeud(self,noeud):
        self.noeuds.add(noeud)
    def addRelation(self, relation, noeud1, noeud2):
        # Ajout des noeuds non-présents dans le graphe
        if noeud1 not in self.noeuds:
            self.addNoeud(noeud1)
        if noeud2 not in self.noeuds:
            self.addNoeud(noeud2)

        # Ajout de la relation
        try:
            self.relations[noeud1, noeud2] |= set(relation)
        except KeyError:
            self.relations[noeud1, noeud2] = set(relation)
        # Ajout de la relation transposée
        try:
            self.relations[noeud2, noeud1] |= set(transpose[relation])
        except KeyError:
            self.relations[noeud2, noeud1] = set(transpose[relation])

    def propagation(self, noeud1, noeud2):
        """ Applique l'algorithme d'Allen pour propager la relation entre 
        `noeud1` et `noeud2` dans le reste du graphe.
        """
        pass

        

In [212]:
from copy import deepcopy
def propagation(g1, n1,n2,relation):
    g = deepcopy(g1)
    pile = []
    pile.append((relation,n1,n2))
    while pile :
        rel,i,j = pile.pop()
        for k in g.get_noueds():
            if k != i and k != j :
                nouv_1 = (g.getRelations(i,k)).intersection(compose_set(g.getRelations(i,j),g.getRelations(j,k)))
                nouv_2 = (g.getRelations(k,j)).intersection(compose_set(g.getRelations(k,i),g.getRelations(i,j)))
                if nouv_1 == set() or nouv_2 == set():
                    print("KO")
                    return {} 
                if nouv_1 != g.getRelations(i,k):
                    pile.append((g.getRelations(i,k),i,k)) 
                if nouv_2 != g.getRelations(k,n2):
                    pile.append((g.getRelations(k,j),k,j)) 
        

    

In [213]:
g = Graphe()
g.addRelation('<','A','B')
g.addRelation('>','A','C')

#propagation(g,'B','C')
g.getRelations('C','A')

{'<'}

In [214]:
propagation(g, 'B','C',{'='})
